In [ ]:
from keras.applications.inception_v3 import InceptionV3
import os
from keras.layers import Flatten,Dense,AveragePooling2D
from keras.models import Model
from keras.optimizers import SGD,RMSprop
from keras.callbacks import ModelCheckpoint,CSVLogger,TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

# import tensorflow as tf
# #制定GPU，限制GPU内存
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
# set_session(tf.Session(config=config))

learning_rate = 0.0001
img_width = 299
img_height = 299
nbr_train_samples = 3019
nbr_validation_samples = 758
nbr_epochs = 25
batch_size = 32

train_data_dir = './train_split'
val_data_dir = './val_split'

FishNames = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
print('Loading InceptionV3 Weights ...')
# InceptionV3_model = None
# if not os.path.isfile('./weights-copy.h5'):
InceptionV3_notop = InceptionV3(include_top=False, weights='imagenet',
                input_tensor=None, input_shape=(299, 299, 3))
# Note that the preprocessing of InceptionV3 is:
# (x / 255 - 0.5) x 2
print('Adding Average Pooling Layer and Softmax Output Layer ...')
output = InceptionV3_notop.get_layer(index = -1).output  # Shape: (8, 8, 2048)
output = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(output)
output = Flatten(name='flatten')(output)
output = Dense(8, activation='softmax', name='predictions')(output)

InceptionV3_model = Model(InceptionV3_notop.input, output)
# else:
#     print("load exsit module........")
#     InceptionV3_model = load_model('./weights-copy.h5')

#InceptionV3_model.summary()
print("begin compile........")
optimizer = SGD(lr = learning_rate, momentum = 0.9, decay = 0.0, nesterov = True)
InceptionV3_model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
# autosave best Model
print("autosave best Model")
best_model_file = "./inception_weights.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_acc', verbose = 1, save_best_only = True)
csv_logger = CSVLogger('inception_train.log')
#将loss ，acc， val_loss ,val_acc记录tensorboard
# tensorboard = TensorBoard(log_dir='./logdir', histogram_freq=1,write_graph=False)
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=10.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)

# this is the augmentation configuration we will use for validation:
# only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        shuffle = True,
        # save_to_dir = '/Users/pengpai/Desktop/python/DeepLearning/Kaggle/NCFM/data/visualization',
        # save_prefix = 'aug',
        classes = FishNames,
        class_mode = 'categorical')

validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle = True,
        #save_to_dir = '/Users/pengpai/Desktop/python/DeepLearning/Kaggle/NCFM/data/visulization',
        #save_prefix = 'aug',
        classes = FishNames,
        class_mode = 'categorical')
InceptionV3_model.fit_generator(generator=train_generator,
                                steps_per_epoch=1000,
                                epochs=nbr_epochs,
                                validation_data=validation_generator,
                                validation_steps=nbr_validation_samples,
                                callbacks = [best_model,csv_logger],
                                verbose=1,
                                initial_epoch=0)
# InceptionV3_model.fit_generator(
#         train_generator,
#         samples_per_epoch = nbr_train_samples,
#         nb_epoch = nbr_epochs,
#         validation_data = validation_generator,
#         nb_val_samples = nbr_validation_samples,
#         callbacks = [best_model,csv_logger,tensorboard])

d:\program files\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Loading InceptionV3 Weights ...
Adding Average Pooling Layer and Softmax Output Layer ...
begin compile........
autosave best Model
Found 3019 images belonging to 8 classes.
Found 758 images belonging to 8 classes.
Epoch 1/25
  13/1000 [..............................] - ETA: 12:30:46 - loss: 1.9225 - acc: 0.2764